In [130]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import pandas as pd

import time

from datetime import date
from dateutil.rrule import rrule, DAILY, MONTHLY
from dateutil.relativedelta import relativedelta

import os.path
import os

import sys

import numpy as np
from io import StringIO

In [2]:
detailsList = []

coinDetails = {}
coinDetails['name'] = 'Bitcoin'
coinDetails['start'] = date(2013, 6, 1)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Dashpay'
coinDetails['start'] = date(2013, 12, 15)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Dogecoin'
coinDetails['start'] = date(2013, 1, 1)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Ethereum'
coinDetails['start'] = date(2015, 8, 8)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Litecoin'
coinDetails['start'] = date(2013, 5, 5)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Monero'
coinDetails['start'] = date(2014, 5, 21)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Ripple'
coinDetails['start'] = date(2013, 8, 5)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Stellar'
coinDetails['start'] = date(2014, 8, 5)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

In [ ]:
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", os.getcwd() + "\\data")
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/json, application/x-download, text/csv, application/csv")

driver = webdriver.Firefox(firefox_profile=profile)

url = 'https://trends.google.com/trends/explore?date={}%20{}&q={}'

for coinDetail in detailsList:
    print("Scraping {} monthly from Google Trends".format(coinDetail['name']))
    
    old = coinDetail['start']
    coinDetail['start'] = coinDetail['start'] + relativedelta(months=1)
    
    for dt in rrule(MONTHLY, dtstart=coinDetail['start'], until=coinDetail['end']):
        
        try:
            goURL = url.format(old.strftime('%Y-%m-%d'), dt.strftime('%Y-%m-%d'), coinDetail['name'])
            time.sleep(3)

            driver.get(goURL)

            try:
                elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'i.material-icons-extended.gray')))
            except TimeoutException:
                sys.exit("Too much time")

            driver.find_elements_by_xpath("//*[contains(text(), 'file_download')]")[0].click()
            time.sleep(2)
            a = 0
            
            while a == 0:
                downloadedFile = '{}\\data/multiTimeline.csv'.format(os.getcwd())

                if (os.path.isfile(downloadedFile)):
                    a = 1
                    
                    try:
                        os.rename(downloadedFile, '{}\\data/{}-{}-{}.csv'.format(os.getcwd(), old.strftime('%Y-%m-%d'), dt.strftime('%Y-%m-%d'), coinDetail['name']))
                    except:
                        os.remove(downloadedFile)
                else:
                    driver.find_element_by_css_selector('i.material-icons-extended.gray').click()
                    print('Re clicking')
                    time.sleep(2)
                    
            old  = dt
        except:
            print("Sleeping 1 minutes")
            time.sleep(60)

Scraping Bitcoin monthly from Google Trends
Scraping Dashpay monthly from Google Trends
Sleeping 1 minutes
Scraping Dogecoin monthly from Google Trends
Scraping Ethereum monthly from Google Trends
Scraping Litecoin monthly from Google Trends
Sleeping 1 minutes
Sleeping 1 minutes
Scraping Monero monthly from Google Trends
Scraping Ripple monthly from Google Trends
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
Scraping Stellar monthly from Google Trends
Sleeping 1 minutes
Sleeping 1 minutes
Sleeping 1 minutes
S

In [185]:
def combined_trends(): 
    directory = os.getcwd() + "\\data"
    files = os.listdir(directory)
    files = pd.Series(files)

    trends = {}

    for coinDetail in detailsList:
        currentFiles = files[files.str.split('-').str[-1] == '{}.csv'.format(coinDetail['name'])]

        df = pd.DataFrame()

        for file in currentFiles:
            fullName = "{}\\{}".format(directory, file)
            details = open(fullName, "r").readlines()[3:]
            st = ''.join(details)

            try:
                tempDf = pd.read_csv(StringIO(st), header=None) 
                df = pd.concat([df, tempDf])
            except: 
                pass #because some file are empty and we removed initial lines

        trends[coinDetail['name']] = df
    
    
    return trends

In [186]:
trends = combined_trends()

In [232]:
def fix_missing(trends):
    
    profile = webdriver.FirefoxProfile()
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.download.dir", os.getcwd() + "\\data")
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/json, application/x-download, text/csv, application/csv")

    driver = webdriver.Firefox(firefox_profile=profile)
    
    for trend in trends:
        df = trends[trend]
        df.columns = ['Date', 'Trend']

        df = df.set_index('Date')
        df = df[~df.index.duplicated(keep='first')]

        realDates = pd.date_range(start=df.index[0], end=df.index[-1])
        
        df.index = pd.to_datetime(df.index)

        missing = realDates.symmetric_difference(df.index)

        if (missing.shape[0] > 0):
            
            if (trend == 'Dashpay'):
                trend = 'Darkcoin'
            
            print("\nMissing in {}".format(trend))
            
            missingVals = missing[0::30]

            for start in missingVals:
                end = start + relativedelta(months=1)
                print("Starting: {} Ending: {}".format(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')))
                
                goURL = url.format(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'), trend)
                
                driver.get(goURL)

                try:
                    elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'i.material-icons-extended.gray')))
                except TimeoutException:
                    sys.exit("Too much time")

                driver.find_elements_by_xpath("//*[contains(text(), 'file_download')]")[0].click()
                time.sleep(2)
                a = 0

                while a == 0:
                    downloadedFile = '{}\\data/multiTimeline.csv'.format(os.getcwd())

                    if (os.path.isfile(downloadedFile)):
                        a = 1

                        try:
                            if (trend == 'Darkcoin'):
                                trend = 'Dashpay'
                                
                            os.rename(downloadedFile, '{}\\data/{}-{}-{}.csv'.format(os.getcwd(), old.strftime('%Y-%m-%d'), dt.strftime('%Y-%m-%d'), coinDetail['name']))
                        except:
                            os.remove(downloadedFile)
                    else:
                        driver.find_element_by_css_selector('i.material-icons-extended.gray').click()
                        print('Re clicking')
                        time.sleep(2)

                old  = dt

In [233]:
fix_missing(trends)


Missing in Bitcoin
Starting: 2018-01-02 Ending: 2018-02-02

Missing in Darkcoin
Starting: 2014-05-16 Ending: 2014-06-16
Starting: 2014-11-16 Ending: 2014-12-16
Starting: 2014-12-16 Ending: 2015-01-16


WebDriverException: Message: Failed to decode response from marionette
